In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import numpy as np

In [1]:
# Define image dimensions and batch size
img_height, img_width = 128, 128
batch_size = 5

# Define directories for training and testing data
train_data_dir = "../dataset/train"
test_data_dir = "../dataset/test"

In [4]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Data augmentation for testing images (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:

# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=5,
    class_mode='categorical',
    shuffle=False  # No need to shuffle for evaluation
)

Found 1174 images belonging to 14 classes.
Found 428 images belonging to 14 classes.


In [5]:

# Define the MobileNet base model
base_model = MobileNet(
    input_shape=(img_height, img_width, 3),  # Adjust input shape
    include_top=False,  # Exclude the fully-connected layers
    weights='imagenet'  # Pre-trained on ImageNet
)

# Freeze the base model layers
base_model.trainable = False


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


In [6]:

# Create the classification head
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(14, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)




Epoch 1/15
235/235 [==============================] - 29s 79ms/step - loss: 1.8506 - accuracy: 0.3893 - val_loss: 1.0801 - val_accuracy: 0.6636
Epoch 2/15
235/235 [==============================] - 17s 72ms/step - loss: 1.1450 - accuracy: 0.6371 - val_loss: 1.0094 - val_accuracy: 0.6682
Epoch 3/15
235/235 [==============================] - 16s 70ms/step - loss: 0.9177 - accuracy: 0.7223 - val_loss: 0.8295 - val_accuracy: 0.7360
Epoch 4/15
235/235 [==============================] - 17s 70ms/step - loss: 0.8178 - accuracy: 0.7453 - val_loss: 0.8471 - val_accuracy: 0.7243
Epoch 5/15
235/235 [==============================] - 16s 70ms/step - loss: 0.7570 - accuracy: 0.7624 - val_loss: 0.8486 - val_accuracy: 0.7570
Epoch 6/15
235/235 [==============================] - 17s 72ms/step - loss: 0.6437 - accuracy: 0.8211 - val_loss: 0.7824 - val_accuracy: 0.7500
Epoch 7/15
235/235 [==============================] - 16s 70ms/step - loss: 0.5932 - accuracy: 0.8118 - val_loss: 0.7931 - val_accuracy:

In [8]:
# predicting train dataset 
test_pridiction = model.predict(test_generator)

# true labels 
test_true_labels = test_generator.classes




86/86 [==============================] - 3s 24ms/step


In [9]:
print( test_pridiction.shape)
print( test_true_labels.shape)




(428, 14)
(428,)


In [10]:
#calculating accuracy
accuracy = accuracy_score(test_true_labels, np.argmax(test_pridiction, axis=1))
print(accuracy)

0.7827102803738317
